In [1]:
import pandas as pd
import numpy as np
import os
import json
import re 
from collections import defaultdict

In [2]:
# 파일 목록
path = "./products"
file_list = os.listdir(path)

json_list = list()
for i in file_list :
    if i.find(".json") != -1 :
        json_list.append(i)
print(json_list)

['productCategories.json', 'products(도서,온라인강의,오프라인강의).json', 'products.json', 'productCurations.json']


# $ 표시 제거

In [3]:
# $ 들어간 거 제거 
for file_name in json_list :
    path2 = path + "/" + file_name
    
    with open(path2 , 'r' , encoding='UTF-8') as f :
        data = json.load(f, strict = False)

    def sol(data):
        if isinstance(data , dict):
            for key , value in data.items():
                if isinstance( value, dict):
                    if len(value.keys()) == 1 :
                        value_key = list(value.keys())[0]
                        if value_key == "$oid" :
                            data[key] = value["$oid"]
                        elif value_key == "$date" :
                            data[key] = value["$date"]
                    else:
                        pass
                elif isinstance( value, list):
                    for idx, v  in enumerate(value):
                        if isinstance(v , dict):
                            if len(v.keys()) == 1 :
                                value_key = list(v.keys())[0]
                                if value_key == "$oid" :
                                    value[idx] = v["$oid"]
                                elif value_key == "$date" :
                                    value[idx] = v["$date"]
                            else:
                                sol(v)
                        else:
                            pass
                    #data[key].append(sol(value))
                elif isinstance( value, str) : # value가 str 일때
                    pass
                if key == "$oid" :
                    return value
                elif key == "$date" :
                    return value
                else :
                    pass
        
    for i in data :
        sol(i)


    with open(path2 , 'w' , encoding='UTF-8' ) as file :
        json.dump(data, file, ensure_ascii = False, indent=4 )

# html 태그 제거

In [4]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

for file_name in json_list :
    path2 = path + "/" + file_name
    
    with open(path2 , 'r' , encoding='UTF-8') as f :
        data = json.load(f, strict = False)

    def sol(data):
        pass
        
    for idx, d in enumerate(data) :
        for key , value in d.items():
            if isinstance(value, str) :
                data[idx][key] = cleanhtml(value)


    with open(path2 , 'w' , encoding='UTF-8' ) as file :
        json.dump(data, file, ensure_ascii = False, indent=4 )

# product 관련 json 각각 속성 파악

In [5]:
print(json_list)

['productCategories.json', 'products(도서,온라인강의,오프라인강의).json', 'products.json', 'productCurations.json']


## productsCategories.json

In [11]:
file_path = "./products/" + json_list[0]
a = defaultdict(list)
with open(file_path, 'r') as file:
    data0 = json.load(file)
data0[0]

{'_id': '5df6eb494267e105dffef92a',
 'date_created': '2019-12-16T02:26:17.072Z',
 'date_updated': '2022-09-14T02:42:46.575Z',
 'image': '',
 'name': '기획서비스',
 'description': '',
 'meta_description': '',
 'meta_title': '',
 'enabled': False,
 'sort': '',
 'parent_id': None,
 'position': 234,
 'slug': 'planning-service'}

In [12]:
df_cate = pd.DataFrame(data0)
df_cate.head(1)

,_id,date_created,date_updated,image,name,description,meta_description,meta_title,enabled,sort,parent_id,position,slug
0,5df6eb494267e105dffef92a,2019-12-16T02:26:17.072Z,2022-09-14T02:42:46.575Z,,기획서비스,,,,False,,None,234,planning-service


## products.json

- category_id 
    - "dasdqwr" 처럼 난수로 되어잇는 값을 name으로 변경 ( e.g. "qwrqwe" -> 도서 )
- category_ids
    - category와 동일

In [9]:
# 파일불러오기
file_path = "./products/" + json_list[2]
a = defaultdict(list)
with open(file_path, 'r') as file:
    data2 = json.load(file)

In [13]:
# category_id 값이 id로 되어있는거 name로 변경
for d in data2:
    try : 
        d['category_id'] = df_cate[df_cate['_id'] == d['category_id']]['name'].values[0]
    except:
        print(df_cate[df_cate['_id'] == d['category_id']]['name'].values, d['category_id'])
        d['category_id'] = ""

[] 5de0bc390a02d105d392c398
[] 5de0bc390a02d105d392c398
[] 5de0bc390a02d105d392c398
[] 5de0bc390a02d105d392c398
[] 5de0bc390a02d105d392c398
[] 5de0bc390a02d105d392c398
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None
[] None


In [116]:
# category_ids (서브) 값을 위와 동일하게 name처리
for d in data2:
    for idx , value in enumerate(d['category_ids']):
        try : 
            d['category_ids'][idx] = df_cate[df_cate['_id'] == value]['name'].values[0]
        except:
            d['category_ids'][idx] = ""